TODO

[github](https://github.com/abhishekkrthakur/commonlit-pairwise-model)

In [1]:
from IPython.display import clear_output, Image
!pip install transformers
clear_output()

In [7]:
import os
import re
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch import nn, optim
import transformers
from numpy import random
from sklearn import model_selection
import itertools

"""
получения информации о запущенных процессах
и использовании системы (ЦП, память, диски, сеть, датчики) в Python.
"""
import psutil

path_tr = '/content/drive/MyDrive/CommonLit/input/train.csv'
path_test = '/content/drive/MyDrive/CommonLit/input/test.csv'
path_sub = '/content/drive/MyDrive/CommonLit/input/sample_submission.csv'

SEED =13
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
def seed_everything(seed: int) -> None:
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


def create_folds(data, num_splits):
    data["kfold"] = -1
    data = data.sample(frac=1).reset_index(drop=True)
    num_bins = int(np.floor(1 + np.log2(len(data))))
    data.loc[:, "bins"] = pd.cut(
        data["target"], bins=num_bins, labels=False
    )
    kf = model_selection.StratifiedKFold(n_splits=num_splits)
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)):
        data.loc[v_, 'kfold'] = f
    data = data.drop("bins", axis=1)
    return data


class CommonlitDataset:
    def __init__(self, excerpts, target_dict, error_dict, tokenizer, max_len, num_samples=None):
        self.excerpts = excerpts
        self.target_dict = target_dict
        self.error_dict = error_dict
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.num_samples = num_samples
        self.count = 0

    def __len__(self):
        if self.num_samples is None:
            return len(self.excerpts)
        return self.num_samples

    def __getitem__(self, item):
        if self.num_samples is not None:
            self.count += 1
            if self.count >= self.num_samples / psutil.cpu_count():
                self.count = 0
                random.shuffle(self.excerpts)

        text1 = str(self.excerpts[item][1])
        text2 = str(self.excerpts[item][0])
        target = [
            self.target_dict[text2],
            self.target_dict[text1],
        ]

        inputs1 = self.tokenizer(text1, max_length=self.max_len, padding="max_length", truncation=True)
        inputs2 = self.tokenizer(text2, max_length=self.max_len, padding="max_length", truncation=True)

        ids1 = inputs1["input_ids"]
        mask1 = inputs1["attention_mask"]

        ids2 = inputs2["input_ids"]
        mask2 = inputs2["attention_mask"]

        return {
            "ids1": torch.tensor(ids1, dtype=torch.long),
            "mask1": torch.tensor(mask1, dtype=torch.long),
            "ids2": torch.tensor(ids2, dtype=torch.long),
            "mask2": torch.tensor(mask2, dtype=torch.long),
            "targets": torch.tensor(target, dtype=torch.float),
        }

In [8]:
seed_everything(42)
df = pd.read_csv(path_tr)
df = create_folds(df, num_splits=5)

In [9]:
df

,id,url_legal,license,excerpt,target,standard_error,kfold
0,b51730f9c,NaN,NaN,"Alice looked at the jury-box, and saw that, in...",-0.432678,0.487498,0
1,4d403fd57,https://en.wikipedia.org/wiki/Artificial_intel...,CC BY-SA 3.0,Artificial intelligence (AI) is intelligence e...,-1.161746,0.458396,0
2,0f789ee41,NaN,NaN,A gruff squire on horseback with shiny top boo...,-2.367914,0.519369,0
3,87f96eb79,NaN,NaN,But that hadn't helped Washington.\nThe Americ...,-0.842596,0.466193,0
4,b9cca6661,NaN,NaN,The principal business of the people of this c...,-0.748452,0.433000,0
...,...,...,...,...,...,...,...
2829,240b89006,NaN,NaN,The steam is supplied by two circular return t...,-3.309178,0.550580,4
2830,243f9f286,NaN,NaN,Living things are different from things that a...,-0.216738,0.484298,4
2831,e044e1347,NaN,NaN,"I'd always longed for adventures. You see, my ...",-1.584384,0.447262,4
2832,7d016018e,NaN,NaN,In these times one dread lies heavy on heart a...,-2.034688,0.466946,4


In [14]:
"""
base string is excerpt where target is 0 in the dataframe
df.loc[df.target == 0, "excerpt"]
>> 
  The sun was shining in a cloudless sky, and no...
"""
base_string = df.loc[df.target == 0, "excerpt"].values[0]
# create dictionary out of excerpt and target columns from dataframe
# text: value
target_dict = dict(zip(df.excerpt.values.tolist(), df.target.values.tolist()))
df_train = df[df.kfold != 0].reset_index(drop=True)
df_valid = df[df.kfold == 0].reset_index(drop=True)

In [ ]:
"""
train_pair:
    ('The soil most suitable for the full development t or ... large yellow flowers.',
    'The journey from Burnet to Hillsover was a very . The ... in fact, notice it.'),
    ('The soil most suitable for the full developmentot or ... large yellow flowers.',
    'One of the first steps of the scientific method stand ...  dark matter (CDM)."'),
    ('The soil most suitable for the full developmentot or ... large yellow flowers.',
    'Bhabhloo was a very, very naughty bear, loved by ...)
    ('The soil most suitable for the full development t or ... large yellow flowers.',
    'In 1832 Dr. James Mease, ... )
valid_pair:
    we use for each base string to each valid    
        ('The sun was shining in  again. ...',
        "Alice looked at the juriness?' the ... ng whatever,' said Alice."),
        ('The sun was shining in  again. ...',
        'Artificial intelligence ...)
"""
training_pairs = list(itertools.combinations(df_train.excerpt.values.tolist(), 2))
# randomize training_pairs
random.shuffle(training_pairs)
validation_pairs = [(base_string, k) for k in df_valid.excerpt.values.tolist()]